In [ ]:
include("../src/HPFEM.jl")

In [ ]:
TF = Float64
nel = 10
nnodes = nel + 1
idir = [1]  ### A: Só um lado é Dirichilet o outro é Neumann
M = 6
Q = M+2
b = HPFEM.Lagrange1d(M, TF)
quad = HPFEM.QuadType(Q, HPFEM.GLJ, TF)
bas = HPFEM.Basis1d(b,quad, TF)
#bas = HPFEM.SEM1d(M)
#bas = HPFEM.Basis1d(M,Q)
lmap = HPFEM.locmap(bas)
dof = HPFEM.DofMap1d(lmap, nnodes, idir);


In [ ]:
k = 2
uexact(x) = sin(2π*k*x)
rhsfun(x) = sin(2π*k*x) * (1.0 + (2π*k)^2)
duexact(x) = 2π*k * cos(2π*k*x)  ### A: Precisamos agora conhecer a derivada (pelo menos no ponto)

In [ ]:
a = 1.7
b = 6.3
nodes = [TF(x) for x in linspace(a, b, nnodes)];

In [ ]:
elems = [HPFEM.Element1d(e, nodes[e], nodes[e+1], bas) for e = 1:nel];

In [ ]:
solver = HPFEM.CholeskySC(dof, HPFEM.BBMatrix1d, TF);

In [ ]:
for e = 1:nel
    Ae = zeros(TF, M, M)
    HPFEM.add_stiff_matrix!(bas, elems[e], Ae)
    HPFEM.add_mass_matrix!(bas, elems[e], Ae)
    HPFEM.add_local_matrix(solver, e, Ae)
end


In [ ]:
Fe = zeros(HPFEM.nmodes(lmap), nel)
bnd = HPFEM.bndidx(lmap)

for e = 1:nel
    fe = rhsfun(elems[e].x)
    HPFEM.add_rhs!(bas, elems[e], fe, sub(Fe, :, e))
end
Fe[bnd[2],nel] += bas(one(TF), bnd[2]) * duexact(b) ### A: Adicionar o termo vdu/dx no lado direito!

# Apply Dirichilet BCs:
Fe[bnd[1],1] = uexact(a);


In [ ]:
nξ = 101
ξ = collect(linspace(-1,1,nξ));
ϕ = zeros(nξ, M)
for i = 1:M
    ϕ[:,i] = bas(ξ, i)
end

Ue = ϕ * Fe;

In [ ]:
using PyPlot
maxerr = 0.0
for e = 1:nel
    el = elems[e]
    x = (1-ξ)*el.a/2 + (1+ξ)*el.b/2 
    uu = uexact(x)
    err = maxabs(uu-Ue[:,e])
    if err > maxerr maxerr = err end
        
    plot(x, Ue[:,e], "r", x, uu, "b")
end
maxerr